In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

import lsst.sims.maf.db as db
import lsst.sims.maf.utils as utils
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.metricBundles as metricBundles

In [2]:
dbFile = 'feature_baseline_10yrs.db'
# This connection checks which version it thinks the database is and should do the right thing schema-wise
conn = db.OpsimDatabase(dbFile)
outDir='temp'
resultsDb = db.ResultsDb(outDir=outDir)

In [4]:
bundleList = []

metric = metrics.BruteOSFMetric()
slicer = slicers.UniSlicer()
sql = ''

bundleList.append(metricBundles.MetricBundle(metric,slicer,sql))

In [5]:
bd = metricBundles.makeBundlesDictFromList(bundleList)
bg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, resultsDb=resultsDb)
bg.runAll()

Querying database SummaryAllProps with no constraint for columns ['visitExposureTime', 'observationStartMJD'].
Found 2304791 visits
Running:  ['opsim_BruteOSFMetric_UNIS']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [8]:
bundleList[0].metricValues

masked_array(data = [0.7054909758573882],
             mask = [False],
       fill_value = -666.0)